In [ ]:
!pip install transformers==4.17

In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = 'distilbert-base-cased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer(data['train'][0]['tokens'], is_split_into_words=True)

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
data['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [ ]:
data['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
data['train'].features['ner_tags'].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
label_names = data['train'].features['ner_tags'].feature.names

In [ ]:
idx = 0
t = tokenizer(data['train'][idx]['tokens'], is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
type(t)
# its not a dictionary, in fact an object

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
t.tokens()

#its a useful instance method .tokens()
#here lamb is made into subwords
#this situation is to be handled differently

#the next task to wrap this into tokenize function that wraps up so that we can apply truncation
# we then call the map function to map the tokenizer to our datasets
#problem is that target is a sequence

#by tokenization the input sequence may be more than target sequence, can be a problem

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

Aligning targets to tokens

For any word split into multiple tokens, we assign the same target

la(5), ##mb(5)(repeat the same target again)

This may called as EXPAND

By this method the length of the input sequence will be equal to the target sequence

What about special tokens [CLS], [SEP] ?
This tokens will also have to be accounted for as tokens

Specifically the hugging face library uses integer value -100 for the outputs and targets that should be ignored

A constant is taken because gradient will be 0, and there will be no weight upadation

Algotrithm outline

We have to loop through each word_id
three scenarios
1 - If word_id is None (label=-100)
2 - IF word_id is integer (new word, grab a label for this word)
3 If word_id is integer (repeated word, assign previous token)

In [ ]:
# IOB
# B - beginning of an entity, I-inside of an entity, O-outside of entity

# if "Steve" = ["Ste", "ve"]
# we want [B-PER, I-PER]

In [ ]:
# ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
begin2inside = {
    1:2,
    3:4,
    5:6,
    7:8,
}

In [ ]:
# Align target function

#def align_targets(labels, word_ids):
#  aligned_labels=[]
#  last_word=None
#  for word in word_ids:
#    if word is None:
#      label=-100
#    elif word != last_word:
#      label=labels[word]
#    else:
#      label=labels[word]

#    aligned_labels.append(label)
#    last_word=word


#  return aligned_labels


In [ ]:
def align_targets(labels, word_ids):
  aligned_labels=[]
  last_word=None
  for word in word_ids:
    if word is None:
      label=-100
    elif word != last_word:
      label=labels[word]
    else:
      label=labels[word]

      if label in begin2inside:
        label = begin2inside[label]

    aligned_labels.append(label)
    last_word=word


  return aligned_labels

In [ ]:
def tokenize_fn(batch):
  tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True)
  labels_batch = batch['ner_tags']
  aligned_label_batch = []
  for i, labels in enumerate(labels_batch):
    word_ids = tokenized_inputs.word_ids(i)
    aligned_label_batch.append(align_targets(labels,word_ids))
  tokenized_inputs['labels'] = aligned_label_batch
  return tokenized_inputs


In [ ]:
tokenized_dataset = data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

# **Target labels**

In [ ]:
t.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [ ]:
t.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [ ]:
labels = data['train'][idx]['ner_tags']
word_ids = t.word_ids()
aligned_targets = align_targets(labels, word_ids)
aligned_targets

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [ ]:
aligned_labels = [label_names[t] if t>=0 else None for t in aligned_targets]
for x, y in zip(t.tokens(), aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
la	O
##mb	O
.	O
[SEP]	None


In [ ]:
# makeup a fake input just to test it

words = ['[CLS]', 'Ger', '##man', 'call', 'to', 'boycott','Micro','##soft', '[SEP]']
word_ids = [None, 0, 0, 1, 2, 3, 4, 4, None]
labels = [7, 0, 0, 0, 3]
aligned_targets = align_targets(labels, word_ids)
aligned_labels = [label_names[t] if t>=0 else None for t in aligned_targets]

for x, y in zip(words, aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
Ger	B-MISC
##man	I-MISC
call	O
to	O
boycott	O
Micro	B-ORG
##soft	I-ORG
[SEP]	None


In [ ]:
# tokenize both inputs and outputs

def tokenize_fn(batch):
  tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True)
  labels_batch = batch['ner_tags']
  aligned_labels_batch = []
  for i, labels in enumerate(labels_batch):
    word_ids = tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels, word_ids))
  tokenized_inputs['labels'] = aligned_labels_batch

  return tokenized_inputs

In [ ]:
data['train'].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [ ]:
tokenized_datasets = data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

# **Data Collater**
# - *Padding is done per batch basis*
# - This is handled by data collator


In [ ]:
from transformers import DataCollatorForTokenClassification

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
tokenized_datasets['train'][0:2]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [ ]:
# although we don't have to use data collator manually, we will see how it works

In [ ]:
[tokenized_datasets['train'][i] for i in range(2)]

[{'input_ids': [101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]},
 {'input_ids': [101, 1943, 14428, 102],
  'attention_mask': [1, 1, 1, 1],
  'labels': [-100, 1, 2, -100]}]

In [ ]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])

In [ ]:
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

# ***Computing Metrics***

In [ ]:
# What happens when we have multiple targets per sample

#one solution - is to flatten all targtes or predictions into a single list and compute metrics if they are all a independent predictions

In [ ]:
!pip install seqeval

# this library has the sole purpose of computing metrics of NLP tasks where the target is a sequence

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("seqeval")

<ipython-input-44-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# this doesn't work

# metric.compute(predictions=[1,0,0], references=[0,0,0])

In [ ]:
metric.compute(predictions=[['A', 'A', 'A']], references=[['A', 'B', 'A']])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encoun

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [ ]:
metric.compute(predictions=[['O', 'O', 'I-ORG', 'B-MISC']], references=[['O', 'B-ORG', 'I-ORG', 'B-MISC']])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 0.5,
 'overall_f1': 0.5,
 'overall_accuracy': 0.75}

In [ ]:
def compute_metric(logits_and_labels):
  logits, labels = logits_and_labels #both logits and labels are of shape nxt, where n is batch size, and t is sequence length
  preds = np.argmax(logits, axis=-1)

  str_labels = [[label_names[t] for t in label if t != -100] for label in labels]

  str_preds = [[label_names[p] for p, t in zip(pred, targ) if t != -100] for pred, targ in zip(preds, labels)]

  # this is called double list comprehension
  the_metrics = metric.compute(predictions=str_preds, references=str_labels)

  return {
      'precision' : the_metrics['overall_precision'],
      'f1' : the_metrics['overall_f1'],
      'recall' : the_metrics['overall_recall'],
      'accuracy' : the_metrics['overall_accuracy']
  }


# **Metrics**

In [ ]:
id2label = {k:v for k, v in enumerate(label_names)}
label2id = {v:k for k, v in id2label.items()}

# **Model and Trainer**

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    'distilbert-finetuned-ner',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay=0.01,
)

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    compute_metrics = compute_metric,
    tokenizer = tokenizer,

)

In [ ]:
import numpy as np

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5268


Epoch,Training Loss,Validation Loss,Precision,F1,Recall,Accuracy
1,0.103500,0.163226,0.855090,0.869747,0.884915,0.965349
2,0.045700,0.181971,0.855252,0.873408,0.892351,0.966483
3,0.026100,0.189059,0.862865,0.881109,0.900142,0.968280


***** Running Evaluation *****
  Num examples = 3453
  Batch size = 8
Saving model checkpoint to distilbert-finetuned-ner/checkpoint-1756
Configuration saved in distilbert-finetuned-ner/checkpoint-1756/config.json
Model weights saved in distilbert-finetuned-ner/checkpoint-1756/pytorch_model.bin
tokenizer config file saved in distilbert-finetuned-ner/checkpoint-1756/tokenizer_config.json
Special tokens file saved in distilbert-finetuned-ner/checkpoint-1756/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3453
  Batch size = 8
Saving model checkpoint to distilbert-finetuned-ner/checkpoint-3512
Configuration saved in distilbert-finetuned-ner/checkpoint-3512/config.json
Model weights saved in distilbert-finetuned-ner/checkpoint-3512/pytorch_model.bin
tokenizer config file saved in distilbert-finetuned-ner/checkpoint-3512/tokenizer_config.json
Special tokens file saved in distilbert-finetuned-ner/checkpoint-3512/special_tokens_map.json
***** Running Evaluation *****
 

TrainOutput(global_step=5268, training_loss=0.08062919089535284, metrics={'train_runtime': 306.2643, 'train_samples_per_second': 137.538, 'train_steps_per_second': 17.201, 'total_flos': 462023079274890.0, 'train_loss': 0.08062919089535284, 'epoch': 3.0})

# ***Save and Reload as Pipeline***

In [ ]:
trainer.save_model('my_saved_model')

Saving model checkpoint to my_saved_model
Configuration saved in my_saved_model/config.json
Model weights saved in my_saved_model/pytorch_model.bin
tokenizer config file saved in my_saved_model/tokenizer_config.json
Special tokens file saved in my_saved_model/special_tokens_map.json


In [ ]:
from transformers import pipeline

In [ ]:
ner = pipeline('token-classification', model='my_saved_model', aggregation_strategy='simple', device=0)

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "vocab_size": 28996

In [ ]:
ner("Bill Gates of the CEO of Microsoft in Seattle, Washington")

[{'entity_group': 'PER',
  'score': 0.9978424,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.9989512,
  'word': 'Microsoft',
  'start': 25,
  'end': 34},
 {'entity_group': 'LOC',
  'score': 0.9992482,
  'word': 'Seattle',
  'start': 38,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99904877,
  'word': 'Washington',
  'start': 47,
  'end': 57}]